# Klasyfikacja tekstu za pomocą BERT i GPT2

In [1]:
from transformers import BertTokenizer, \
    BertForSequenceClassification, \
    Trainer, \
    TrainingArguments, \
    DataCollatorWithPadding, \
    pipeline
from peft import PeftModel, \
    PeftConfig, \
    LoraConfig, \
    TaskType, \
    get_peft_model
from datasets import load_dataset
from sklearn.metrics import accuracy_score, \
    f1_score, \
    precision_score, \
    recall_score
from sklearn.manifold import TSNE
import plotly.express as px
import numpy as np
import pandas as pd
import torch

## BertForSequenceClassification

In [2]:
dataset = load_dataset(path = 'json', data_files = '../task_1/data/full_text_classification.jsonl')
dataset['train']

Dataset({
    features: ['text', 'label'],
    num_rows: 4441
})

In [ ]:
class BertSequence():
    def __init__(self, model_name, use_lora = False):
        self.convert_dict = {
            'pozytywny wydźwięk': 0, 
            'neutralny wydźwięk': 1, 
            'negatywny wydźwięk': 2, 
            'mowa nienawiści': 3
        }
        
        self.model_name = model_name
        self.use_lora = use_lora
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model = self._load_model()
        self.tokenizer = self._load_tokenizer()
        
        self.dataset = None
        self.train_dataset = None
        self.val_dataset = None
        self.train_tokenized = None
        self.val_tokenized = None
        self.trainer = None
        self.sentence_embeddings = None
        self.tsne_df = None
    
    def _load_model(self):
        try:
            base_model = BertForSequenceClassification.from_pretrained(
                self.model_name, num_labels = len(self.convert_dict)
            )
            if self.use_lora:
                lora_config = LoraConfig(task_type = TaskType.SEQ_CLS, 
                                         r = 64, 
                                         lora_alpha = 1, 
                                         lora_dropout = 0.1)
                base_model = get_peft_model(model = base_model, peft_config = lora_config)
            return base_model.to(self.device)
        
        except Exception as e:
            print("Error loading model:", e)
            return None
        
    def _load_tokenizer(self):
        try:
            return BertTokenizer.from_pretrained(self.model_name)
        except Exception as e:
            print("Error loading tokenizer:", e)
            return None

    def load_data(self, dataset):
        self.dataset = dataset
        return self.dataset
    
    def _convert_labels(self, text):
        text['label'] = self.convert_dict.get(text['label'], -1)
        return text

    def split_data(self, test_size = 0.2):  #, sample_size = 100
        if self.dataset is None:
            raise ValueError("Dataset is not loaded.")
        
        dataset_to_split = self.dataset['train'].train_test_split(test_size = test_size)
        self.train_dataset = dataset_to_split['train'].map(self._convert_labels)
        self.val_dataset = dataset_to_split['test'].map(self._convert_labels)
        
        return self.train_dataset, self.val_dataset
    
    def _tokenize(self, batch):
        return self.tokenizer(batch['text'], padding = 'max_length', truncation = True, max_length = 128)

    def tokenize_data(self):
        if not self.train_dataset or not self.val_dataset:
            raise ValueError("Train/validation datasets are not initialized.")
        
        self.train_tokenized = self.train_dataset.map(self._tokenize, batched = True)
        self.val_tokenized = self.val_dataset.map(self._tokenize, batched = True)
        
        self.train_tokenized.set_format('torch', columns = ['input_ids', 'attention_mask', 'label'])
        self.val_tokenized.set_format('torch', columns = ['input_ids', 'attention_mask', 'label'])
        
        return self.train_tokenized, self.val_tokenized

    def _compute_metrics(self, pred):
        labels = pred.label_ids
        preds = np.argmax(pred.predictions, axis = 1)
        accuracy = accuracy_score(labels, preds)
        f1 = f1_score(labels, preds, average = 'weighted')
        precision = precision_score(labels, preds, average = 'weighted', zero_division = np.nan)
        recall = recall_score(labels, preds, average='weighted', zero_division = np.nan)
        return {'accuracy': accuracy, 'f1': f1, 'precision': precision, 'recall': recall}
    
    def create_trainer(self, output_dir = './results'):
        if not self.train_tokenized or not self.val_tokenized:
            raise ValueError("Tokenized datasets are not initialized.")
        
        training_args = TrainingArguments(
            output_dir = output_dir,
            eval_strategy = 'epoch',
            learning_rate = 2e-3,
            per_device_train_batch_size = 4,
            per_device_eval_batch_size = 4,
            num_train_epochs = 10,
            weight_decay = 0.01,
            logging_dir = './logs',
        )
        
        self.trainer = Trainer(
            model = self.model,
            args = training_args,
            train_dataset = self.train_tokenized,
            eval_dataset = self.val_tokenized,
            data_collator = DataCollatorWithPadding(tokenizer = self.tokenizer),
            compute_metrics = self._compute_metrics,
        )
        
        return self
    
    def train(self):
        if self.trainer:
            return self.trainer.train()
        else:
            raise ValueError("Trainer is not initialized.")
    
    def evaluate(self):
        if self.trainer:
            return self.trainer.evaluate()
        else:
            raise ValueError("Trainer is not initialized.")

    def _create_embeddings(self):
        inputs = self.tokenizer(self.val_dataset['text'], 
                                padding = True, 
                                truncation = True, 
                                max_length = 128, 
                                return_tensors = 'pt').to(self.device)
        with torch.no_grad():
            outputs = self.model(**inputs, output_hidden_states = True)
            last_hidden_states = outputs.hidden_states[-1]
        self.sentence_embeddings = last_hidden_states[:, 0, :].cpu()
    
    def plot_embeddings(self):
        self._create_embeddings()

        tsne = TSNE(n_components = 2, random_state = 42, perplexity = 2)
        tsne_results = tsne.fit_transform(self.sentence_embeddings)

        self.tsne_df = pd.DataFrame(tsne_results, columns = ['x', 'y'])
        label_mapping = {v: k for k, v in self.convert_dict.items()}
        self.tsne_df['label'] = [label_mapping[label] for label in self.val_dataset['label']]
        self.tsne_df['text'] = self.val_dataset['text']

        color_map = px.colors.qualitative.Vivid[:len(self.convert_dict)]
        
        fig = px.scatter(
            self.tsne_df, x = 'x', y = 'y', color = 'label',
            title = 'Wizualizacja osadzeń SentenceBERT przy użyciu t-SNE',
            labels = {'label': 'Wydźwięk'},
            hover_name = self.tsne_df['text'],
            color_discrete_sequence = color_map
        )
        
        fig.update_traces(marker = dict(size = 10), selector = dict(mode = 'markers'))
        fig.show()

In [10]:
def run_model(model_name, dataset, use_lora = False):
    model = BertSequence(model_name, use_lora)
    model.load_data(dataset)
    train, test = model.split_data()
    train_tokenized, test_tokenized = model.tokenize_data()
    trainer = model.create_trainer()
    trainer.train()
    # metrics = trainer.evaluate()
    # print(metrics)
    model.plot_embeddings()
    
    return None

In [11]:
run_model('dkleczek/bert-base-polish-uncased-v1', dataset, use_lora = True)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dkleczek/bert-base-polish-uncased-v1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/531M [00:00<?, ?B/s]

Map:   0%|          | 0/3552 [00:00<?, ? examples/s]

Map:   0%|          | 0/889 [00:00<?, ? examples/s]

Map:   0%|          | 0/3552 [00:00<?, ? examples/s]

...

{'eval_loss': 1.658768892288208, 'eval_accuracy': 0.6794150731158605, 'eval_f1': 0.6808398636603972, 'eval_precision': 0.6837773292938014, 'eval_recall': 0.6794150731158605, 'eval_runtime': 5.2417, 'eval_samples_per_second': 169.602, 'eval_steps_per_second': 42.543, 'epoch': 8.0}
{'loss': 0.1804, 'grad_norm': 0.2809048295021057, 'learning_rate': 0.0003108108108108108, 'epoch': 8.45}


  0%|          | 0/223 [00:00<?, ?it/s]

{'eval_loss': 1.786131739616394, 'eval_accuracy': 0.6850393700787402, 'eval_f1': 0.6856838653630779, 'eval_precision': 0.6871661056190472, 'eval_recall': 0.6850393700787402, 'eval_runtime': 5.2394, 'eval_samples_per_second': 169.675, 'eval_steps_per_second': 42.562, 'epoch': 9.0}
{'loss': 0.1866, 'grad_norm': 0.6911212205886841, 'learning_rate': 0.0001981981981981982, 'epoch': 9.01}
{'loss': 0.1471, 'grad_norm': 7.388421535491943, 'learning_rate': 8.558558558558558e-05, 'epoch': 9.57}


  0%|          | 0/223 [00:00<?, ?it/s]

{'eval_loss': 1.8064097166061401, 'eval_accuracy': 0.6827896512935883, 'eval_f1': 0.6843783742826413, 'eval_precision': 0.6869898535708214, 'eval_recall': 0.6827896512935883, 'eval_runtime': 5.2259, 'eval_samples_per_second': 170.115, 'eval_steps_per_second': 42.672, 'epoch': 10.0}
{'train_runtime': 576.8707, 'train_samples_per_second': 61.574, 'train_steps_per_second': 15.393, 'train_loss': 0.5065217576585375, 'epoch': 10.0}


In [ ]:
import torch
from transformers import AutoTokenizer, GPT2ForSequenceClassification, GPT2Tokenizer

class GPT2Sequence():
    def __init__(self, model_name, use_lora = False):
        self.convert_dict = {
            'pozytywny wydźwięk': 0, 
            'neutralny wydźwięk': 1, 
            'negatywny wydźwięk': 2, 
            'mowa nienawiści': 3
        }
        
        self.model_name = model_name
        self.use_lora = use_lora
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model = self._load_model()
        self.tokenizer = self._load_tokenizer()
        
        self.dataset = None
        self.train_dataset = None
        self.val_dataset = None
        self.train_tokenized = None
        self.val_tokenized = None
        self.trainer = None
        self.sentence_embeddings = None
        self.tsne_df = None
    
    def _load_model(self):
        try:
            base_model = GPT2ForSequenceClassification.from_pretrained(
                self.model_name, num_labels = len(self.convert_dict)
            )
            if self.use_lora:
                lora_config = LoraConfig(task_type = TaskType.SEQ_CLS, 
                                         r = 64, 
                                         lora_alpha = 1, 
                                         lora_dropout = 0.1)
                base_model = get_peft_model(model = base_model, peft_config = lora_config)
            return base_model.to(self.device)
        
        except Exception as e:
            print("Error loading model:", e)
            return None
        
    def _load_tokenizer(self):
        try:
            return GPT2Tokenizer.from_pretrained(self.model_name)
        except Exception as e:
            print("Error loading tokenizer:", e)
            return None

    def load_data(self, dataset):
        self.dataset = dataset
        return self.dataset
    
    def _convert_labels(self, text):
        text['label'] = self.convert_dict.get(text['label'], -1)
        return text

    def split_data(self, test_size = 0.2):  #, sample_size = 100
        if self.dataset is None:
            raise ValueError("Dataset is not loaded.")
        
        dataset_to_split = self.dataset['train'].train_test_split(test_size = test_size)
        self.train_dataset = dataset_to_split['train'].map(self._convert_labels)
        self.val_dataset = dataset_to_split['test'].map(self._convert_labels)
        
        return self.train_dataset, self.val_dataset
    
    def _tokenize(self, batch):
        return self.tokenizer(batch['text'], padding = 'max_length', truncation = True, max_length = 128)

    def tokenize_data(self):
        if not self.train_dataset or not self.val_dataset:
            raise ValueError("Train/validation datasets are not initialized.")
        
        self.train_tokenized = self.train_dataset.map(self._tokenize, batched = True)
        self.val_tokenized = self.val_dataset.map(self._tokenize, batched = True)
        
        self.train_tokenized.set_format('torch', columns = ['input_ids', 'attention_mask', 'label'])
        self.val_tokenized.set_format('torch', columns = ['input_ids', 'attention_mask', 'label'])
        
        return self.train_tokenized, self.val_tokenized

    def _compute_metrics(self, pred):
        labels = pred.label_ids
        preds = np.argmax(pred.predictions, axis = 1)
        accuracy = accuracy_score(labels, preds)
        f1 = f1_score(labels, preds, average = 'weighted')
        precision = precision_score(labels, preds, average = 'weighted', zero_division = np.nan)
        recall = recall_score(labels, preds, average='weighted', zero_division = np.nan)
        return {'accuracy': accuracy, 'f1': f1, 'precision': precision, 'recall': recall}
    
    def create_trainer(self, output_dir = './results_gpt'):
        if not self.train_tokenized or not self.val_tokenized:
            raise ValueError("Tokenized datasets are not initialized.")
        
        training_args = TrainingArguments(
            output_dir = output_dir,
            eval_strategy = 'epoch',
            learning_rate = 2e-3,
            per_device_train_batch_size = 4,
            per_device_eval_batch_size = 4,
            num_train_epochs = 10,
            weight_decay = 0.01,
            logging_dir = './logs',
        )
        
        self.trainer = Trainer(
            model = self.model,
            args = training_args,
            train_dataset = self.train_tokenized,
            eval_dataset = self.val_tokenized,
            data_collator = DataCollatorWithPadding(tokenizer = self.tokenizer),
            compute_metrics = self._compute_metrics,
        )
        
        return self
    
    def train(self):
        if self.trainer:
            return self.trainer.train()
        else:
            raise ValueError("Trainer is not initialized.")
    
    def evaluate(self):
        if self.trainer:
            return self.trainer.evaluate()
        else:
            raise ValueError("Trainer is not initialized.")

    def _create_embeddings(self):
        inputs = self.tokenizer(self.val_dataset['text'], 
                                padding = True, 
                                truncation = True, 
                                max_length = 128, 
                                return_tensors = 'pt').to(self.device)
        with torch.no_grad():
            outputs = self.model(**inputs, output_hidden_states = True)
            last_hidden_states = outputs.hidden_states[-1]
        self.sentence_embeddings = last_hidden_states[:, 0, :].cpu()
    
    def plot_embeddings(self):
        self._create_embeddings()

        tsne = TSNE(n_components = 2, random_state = 42, perplexity = 2)
        tsne_results = tsne.fit_transform(self.sentence_embeddings)

        self.tsne_df = pd.DataFrame(tsne_results, columns = ['x', 'y'])
        label_mapping = {v: k for k, v in self.convert_dict.items()}
        self.tsne_df['label'] = [label_mapping[label] for label in self.val_dataset['label']]
        self.tsne_df['text'] = self.val_dataset['text']

        color_map = px.colors.qualitative.Vivid[:len(self.convert_dict)]
        
        fig = px.scatter(
            self.tsne_df, x = 'x', y = 'y', color = 'label',
            title = 'Wizualizacja osadzeń SentenceGPT2 przy użyciu t-SNE',
            labels = {'label': 'Wydźwięk'},
            hover_name = self.tsne_df['text'],
            color_discrete_sequence = color_map
        )
        
        fig.update_traces(marker = dict(size = 10), selector = dict(mode = 'markers'))
        fig.show()

In [4]:
def run_model2(model_name, dataset, use_lora = False):
    model = GPT2Sequence(model_name, use_lora)
    model.load_data(dataset)
    train, test = model.split_data()
    train_tokenized, test_tokenized = model.tokenize_data()
    trainer = model.create_trainer()
    trainer.train()
    # metrics = trainer.evaluate()
    # print(metrics)
    model.plot_embeddings()
    
    return None

In [5]:
torch.cuda.empty_cache()

In [6]:
torch.device('cuda' if torch.cuda.is_available() else 'cpu')

device(type='cuda')

In [7]:
run_model2('nie3e/sentiment-polish-gpt2-small', dataset, use_lora = True)

c:\Users\20meh\anaconda3\envs\NLP\lib\site-packages\peft\tuners\lora\layer.py:1150: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


Map:   0%|          | 0/3552 [00:00<?, ? examples/s]

Map:   0%|          | 0/889 [00:00<?, ? examples/s]

Map:   0%|          | 0/3552 [00:00<?, ? examples/s]

Map:   0%|          | 0/889 [00:00<?, ? examples/s]

...

{'eval_loss': 2.154465436935425, 'eval_accuracy': 0.6591676040494938, 'eval_f1': 0.6601210023854296, 'eval_precision': 0.6666453581072639, 'eval_recall': 0.6591676040494938, 'eval_runtime': 7.3117, 'eval_samples_per_second': 121.587, 'eval_steps_per_second': 60.862, 'epoch': 8.0}
{'loss': 0.3873, 'grad_norm': 0.002278523053973913, 'learning_rate': 0.00036711711711711714, 'epoch': 8.16}
{'loss': 0.358, 'grad_norm': 0.00018282837118022144, 'learning_rate': 0.0003108108108108108, 'epoch': 8.45}
{'loss': 0.3667, 'grad_norm': 3.6958768367767334, 'learning_rate': 0.00025450450450450454, 'epoch': 8.73}


  0%|          | 0/445 [00:00<?, ?it/s]

{'eval_loss': 2.6419312953948975, 'eval_accuracy': 0.6659167604049494, 'eval_f1': 0.6684250070769266, 'eval_precision': 0.6789349671420387, 'eval_recall': 0.6659167604049494, 'eval_runtime': 7.2279, 'eval_samples_per_second': 122.996, 'eval_steps_per_second': 61.567, 'epoch': 9.0}
{'loss': 0.3624, 'grad_norm': 0.01617749035358429, 'learning_rate': 0.0001981981981981982, 'epoch': 9.01}
{'loss': 0.3263, 'grad_norm': 0.00013825162022840232, 'learning_rate': 0.00014189189189189188, 'epoch': 9.29}
{'loss': 0.2924, 'grad_norm': 18.87890625, 'learning_rate': 8.558558558558558e-05, 'epoch': 9.57}
{'loss': 0.2645, 'grad_norm': 0.0017349281115457416, 'learning_rate': 2.927927927927928e-05, 'epoch': 9.85}


  0%|          | 0/445 [00:00<?, ?it/s]

{'eval_loss': 2.5819807052612305, 'eval_accuracy': 0.672665916760405, 'eval_f1': 0.6725826501568952, 'eval_precision': 0.6746002486912401, 'eval_recall': 0.672665916760405, 'eval_runtime': 7.1948, 'eval_samples_per_second': 123.561, 'eval_steps_per_second': 61.85, 'epoch': 10.0}
{'train_runtime': 863.5818, 'train_samples_per_second': 41.131, 'train_steps_per_second': 20.566, 'train_loss': 0.7655113729270728, 'epoch': 10.0}
